# Extracting data from MovieAPI DataBase
We realize that there is a limit but worked around to find that MovieAPI does a daily dump for us

In [2]:
# For data processing
import numpy as np
import pandas as pd

# For API usage
import requests as r

#our lifesaver, the progress bar
from tqdm import tqdm

In [3]:
#API Key 
API_KEY = 'b4c353d61e8791dac39e77e69a2c08d3' 

##This is to extract our the whole database from their daily dump
simple_extract = pd.read_json('datasets/movie_ids_03_25_2023.json', lines=True)
print(simple_extract.dtypes)
print(simple_extract.head(5))
 

adult                bool
id                  int64
original_title     object
popularity        float64
video                bool
dtype: object
   adult     id                      original_title  popularity  video
0  False   3924                             Blondie       3.502  False
1  False   6124                 Der Mann ohne Namen       0.600  False
2  False   8773                 L'Amour à vingt ans       2.722  False
3  False  25449  New World Disorder 9: Never Enough       1.097  False
4  False  31975      Sesame Street: Elmo Loves You!       0.600   True


In [31]:
cleanedSimple_data = simple_extract.drop(simple_extract[(simple_extract['adult'] == True) | (simple_extract['video'] == True)].index)
print(cleanedSimple_data.head(5))

#Verify adult videos and videos are false
print(cleanedSimple_data['adult'].unique())
print(cleanedSimple_data['video'].unique())

   adult     id                      original_title  popularity  video
0  False   3924                             Blondie       3.502  False
1  False   6124                 Der Mann ohne Namen       0.600  False
2  False   8773                 L'Amour à vingt ans       2.722  False
3  False  25449  New World Disorder 9: Never Enough       1.097  False
5  False      2                               Ariel       9.553  False
[False]
[False]


In [36]:
#We'll only need our ID column for this to create our actual dataset
movie_data = cleanedSimple_data.drop(columns=['adult','video']) 
movie_data = movie_data.sort_values(by=['id'])

#Drop movies with less than 40 popularity
#movie_data = movie_data.drop(movie_data[movie_data['popularity'] <= 8].index)
movie_data = movie_data.drop(movie_data[movie_data['popularity'] <= 80].index)
movie_data = movie_data.reset_index(drop=True)
#print(movie_data.head(5))
print(movie_data.count())
#print(movie_data.tail(5))

id                568
original_title    568
popularity        568
dtype: int64


In [38]:
#movie ID gets stats of movie
#credit get 

#Get Movie Stats through MovieID
cols = ['release_date','vote_average', 'popularity', 'budget','revenue']
for i in tqdm(range(len(movie_data))):
    j = movie_data.iloc[i]['id']
    resp = r.get(f'https://api.themoviedb.org/3/movie/{i}?api_key={API_KEY}')
    try:
        budget = resp.json()['budget']
        print(budget)
        if(budget == 0):
            movie_data.iloc[i]['budget'] = np.nan
        else:
            movie_data.iloc[i]['budget'] = budget
    except KeyError:
        print('bru')
        movie_data.iloc[i]['budget'] = np.nan
    try:
        revenue = resp.json()['revenue']
        if(revenue == 0):
            movie_data.iloc[i]['revenue'] = np.nan
        else:
            movie_data.iloc[i]['revenue'] = revenue
    except KeyError:
        movie_data.iloc[i]['revenue'] = np.nan
    try:
        release = resp.json()['release_date'][0:4]
        movie_data.iloc[i]['release_date'] = release
    except:
        movie_data.iloc[i]['release_date'] = np.nan
    
    try:
        remaining = resp.json()['popularity']
        if(remaining == 0):
            movie_data.iloc[i]['popularity'] = np.nan
        else:
            movie_data.iloc[i]['popularity'] = remaining
    except:
        movie_data.iloc[i]['popularity'] = np.nan

    try:
        voteA = resp.json()['vote_average']
        if(voteA == 0):
            movie_data['vote_average'] = np.nan
        else:
            movie_data['vote_average'] = voteA
    except:
        movie_data['vote_average'] = np.nan

print(movie_data.head(5))


  0%|          | 0/568 [00:00<?, ?it/s]C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['budget'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['revenue'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

bru


C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['budget'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['revenue'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['

bru
0
0


C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['budget'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['revenue'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['

...

bru


C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['budget'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['revenue'] = np.nan
C:\Users\mdsya\AppData\Local\Temp\ipykernel_21116\3211038590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data.iloc[i]['

bru


  6%|▌         | 32/568 [00:08<02:28,  3.60it/s]


KeyboardInterrupt: 

In [26]:
movie_data.to_csv('datasets/test.csv')